> https://docs.langchain.com/oss/python/langgraph/add-memory

# Add short-term memory

In [1]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, MessagesState, START
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat", temperature=0.6)

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

checkpointer = InMemorySaver()  

builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_edge(START, "call_model")

graph = builder.compile(checkpointer=checkpointer)

config = {
    "configurable": {
        "thread_id": "1"
    }
}

graph.invoke({"messages": [{"role": "user", "content": "hi! i am Bob"}]}, config)

{'messages': [HumanMessage(content='hi! i am Bob', additional_kwargs={}, response_metadata={}, id='21483160-0c57-4d32-abc3-a45eef0ccb69'),
  AIMessage(content='Hi Bob! Nice to meet you. 😊  \nIs there anything I can help you with today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 9, 'total_tokens': 30, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'bb4b1281-d06e-49ef-919e-77a41d532239', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--aeb74e1a-62ea-4e18-8059-bbc9164b93df-0', usage_metadata={'input_tokens': 9, 'output_tokens': 21, 'total_tokens': 30, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})]}

## Use in subgraphs

If your graph contains subgraphs, you only need to provide the checkpointer when compiling the parent graph. LangGraph will automatically propagate the checkpointer to the child subgraphs.

In [2]:
from langgraph.graph import START, StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict

class State(TypedDict):
    foo: str

# Subgraph
def subgraph_node_1(state: State):
    return {"foo": state["foo"] + "bar"}

subgraph_builder = StateGraph(State)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph = subgraph_builder.compile()  

# Parent graph
builder = StateGraph(State)
builder.add_node("node_1", subgraph)  
builder.add_edge(START, "node_1")

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

# Add long-term memory

Use long-term memory to store user-specific or application-specific data across conversations.

In [3]:
from langgraph.store.memory import InMemoryStore  
from langgraph.graph import StateGraph

class FooState(TypedDict):
    foo: str

def foo_node(state: FooState):
    return {"foo": state["foo"]}

builder = StateGraph(FooState)
builder.add_node(foo_node)
builder.add_edge(START, "foo_node")
builder.add_edge("foo_node", END)

store = InMemoryStore()
graph = builder.compile(store=store)

## Use sementic search

In [ ]:
from langchain.embeddings import init_embeddings
from langgraph.store.memory import InMemoryStore

# Create store with semantic search enabled
embeddings = init_embeddings("openai:text-embedding-3-small")
store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,
    }
)

store.put(("user_123", "memories"), "1", {"text": "I love pizza"})
store.put(("user_123", "memories"), "2", {"text": "I am a plumber"})

items = store.search(
    ("user_123", "memories"), query="I'm hungry", limit=1
)

# Manage short-term memory

## Trim messages

In [5]:
from langchain_core.messages.utils import (
    trim_messages,  
    count_tokens_approximately  
)
from langchain_deepseek import ChatDeepSeek
from langgraph.graph import StateGraph, START, MessagesState

model = ChatDeepSeek(model="deepseek-chat", temperature=0.6)
summarization_model = model.bind(max_tokens=128)

def call_model(state: MessagesState):
    messages = trim_messages(  
        state["messages"],
        strategy="last",
        token_counter=count_tokens_approximately,
        max_tokens=128,
        start_on="human",
        end_on=("human", "tool"),
    )
    print(messages)
    response = model.invoke(messages)
    return {"messages": [response]}

checkpointer = InMemorySaver()
builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_edge(START, "call_model")
graph = builder.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}
graph.invoke({"messages": "hi, my name is bob"}, config)
graph.invoke({"messages": "write a short poem about cats"}, config)
graph.invoke({"messages": "now do the same but for dogs"}, config)
final_response = graph.invoke({"messages": "what's my name?"}, config)

final_response["messages"][-1].pretty_print()

[HumanMessage(content='hi, my name is bob', additional_kwargs={}, response_metadata={}, id='fd27e5f0-fedf-41f8-a972-f17f7eb67fa9')]
[HumanMessage(content='hi, my name is bob', additional_kwargs={}, response_metadata={}, id='fd27e5f0-fedf-41f8-a972-f17f7eb67fa9'), AIMessage(content="Hi Bob! It's nice to meet you. 😊\n\nIs there anything I can help you with today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 10, 'total_tokens': 33, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 10}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '1e972a5e-8ebe-4c38-81de-6fda0570e449', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ecfbd90f-d96a-4041-8643-a518ad0df9a1-0', usage_metadata={'input_tokens': 10, 'output_tokens': 23, 'total_t

## Delete messages

In [6]:
from langchain.messages import RemoveMessage  

def delete_messages(state):
    messages = state["messages"]
    print(len(messages))
    if len(messages) > 2:
        # remove the earliest two messages
        return {"messages": [RemoveMessage(id=m.id) for m in messages[:2]]}  

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

builder = StateGraph(MessagesState)
builder.add_sequence([call_model, delete_messages])
builder.add_edge(START, "call_model")

checkpointer = InMemorySaver()
app = builder.compile(checkpointer=checkpointer)

for event in app.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    config,
    stream_mode="values"
):
    print([(message.type, message.content) for message in event["messages"]])

for event in app.stream(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
    stream_mode="values"
):
    print([(message.type, message.content) for message in event["messages"]])

[('human', "hi! I'm bob")]
[('human', "hi! I'm bob"), ('ai', "Hi Bob! Nice to meet you. 😊\n\nIs there anything I can help you with today? Whether it's answering questions, solving a problem, or just having a chat — I'm here for it!")]
2
[('human', "hi! I'm bob"), ('ai', "Hi Bob! Nice to meet you. 😊\n\nIs there anything I can help you with today? Whether it's answering questions, solving a problem, or just having a chat — I'm here for it!"), ('human', "what's my name?")]
[('human', "hi! I'm bob"), ('ai', "Hi Bob! Nice to meet you. 😊\n\nIs there anything I can help you with today? Whether it's answering questions, solving a problem, or just having a chat — I'm here for it!"), ('human', "what's my name?"), ('ai', 'You told me your name is **Bob**! 😊  \nUnless you’d like me to call you something else?')]
4
[('human', "what's my name?"), ('ai', 'You told me your name is **Bob**! 😊  \nUnless you’d like me to call you something else?')]


## Summarize messages

In [10]:
from langgraph.graph import MessagesState

class State(MessagesState):
    summary: str

def summarize_conversation(state: State):
    # First, we get any existing summary
    summary = state.get("summary", "")

    # Create our summarization prompt
    if summary:
        # A summary already exists
        summary_message = (
            f"This is a summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

## Manage checkpoints